In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# pass rescale to normalize data 
train_datagen = ImageDataGenerator(rescale=1./255)

# always point to main directory and not subdirectories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(300, 300),    # images all need to be the same size to train
    batch_size=128,
    class_mode='binary'
)

In [ ]:
# pass rescale to normalize data 
test_datagen = ImageDataGenerator(rescale=1./255)

# always point to main directory and not subdirectories
val_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(300, 300),    # images all need to be the same size to train
    batch_size=128,
    class_mode='binary'
)

In [ ]:
model = tf.keras.mode.Sequential([
    # convolutions and pooling
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(300, 300, 3)),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001), metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_generator,     # streams the images from the training directory
    steps_per_epoch=8,   # 1024 images in the training directory, we are doing 128 at a time, need 8 batches to load them all
    epochs=15,
    validation_data=validation_generator,
    validation_steps=8,  # 256 images and handle them in batches of 32, so use 8 as well
    verbose=2
)

## Predicting images

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
    path = '/content/' + fn
    img = image.load_image(path, target_size=(300, 300))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    images = np.vstack([x])
    predictions = model.predict(images, batch_size=10)
    print(predictions[0])
    if predictions[0] > 0.5:
        print(f'{fn} is a human')
    else 
        print(f'{fn} is a horse')